# SQL Parte 2:

## Objetivos:
    1. Conectarnos a una base de datos desde Python.
    2. Enviar un query a la base de datos y exportarlo como un dataframe de pandas.
    3. Introducción a joins entre tablas.
        3.1 Teoría de Conjuntos
        3.2 Joins.
    4. Consideraciones: Subqueries. 
    
 
 
 ### 1. Conectarnos a una base de datos desde Python. 
     
En las sesiones anteriores, emulamos la conexión a una base de datos relacional importando un csv. En esta ocasión, utilizaremos el datasest _pseudo-real_ de employees. Este dataset se ha utilizado para pruebas de _benchmarking_ tanto en análisis como en desempeño.  Nosotros lo utilizaremos para establecer una conexión remota y para introducir algunas consideraciones sobre cómo hacer que nuestros queries sean más eficientes.

Pueden encontrar más información del mismo acá: 

https://relational.fit.cvut.cz/dataset/Employee
https://www.percona.com/blog/2011/02/01/sample-datasets-for-benchmarking-and-testing/

Para realizar esta conexión, utilizaremos la librería *mysql.connector*, la cual pueden instalar en el ambiente del curso por medio de Anaconda. 



In [2]:
import pandas as pd

Creamos una función la cual tendrá como parámetros lo necesario para poder establecer la conexión al SQL:

In [3]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [4]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


### 2. Enviar un query a la base de datos y obtenerlo como un dataframe de Pandas:

    Utilizaremos la función read_sql de pandas para obtener los resultados de nuestros querys: 

In [4]:
query = "SHOW TABLES from employee;"
tables = pd.read_sql(query,connection)
tables

,Tables_in_employee
0,departments
1,dept_emp
2,dept_manager
3,employees
4,salaries
5,titles


In [5]:
query = "select * from employee.employees limit 10;"
employees = pd.read_sql(query,connection)

In [6]:
employees

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [11]:
query = "select count(distinct emp_no) from employee.salaries limit 10;"
cuantos = pd.read_sql(query,connection)
cuantos

,count(distinct emp_no)
0,300024


In [7]:
query = "select * from employee.salaries limit 10;"
salaries = pd.read_sql(query,connection)
salaries

,emp_no,salary,from_date,to_date
0,10001,60117,1986-06-26,1987-06-26
1,10001,62102,1987-06-26,1988-06-25
2,10001,66074,1988-06-25,1989-06-25
3,10001,66596,1989-06-25,1990-06-25
4,10001,66961,1990-06-25,1991-06-25
5,10001,71046,1991-06-25,1992-06-24
6,10001,74333,1992-06-24,1993-06-24
7,10001,75286,1993-06-24,1994-06-24
8,10001,75994,1994-06-24,1995-06-24
9,10001,76884,1995-06-24,1996-06-23


In [8]:
query = "select * from employee.titles limit 10;"
titles = pd.read_sql(query,connection)
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
5,10005,Senior Staff,1996-09-12,9999-01-01
6,10005,Staff,1989-09-12,1996-09-12
7,10006,Senior Engineer,1990-08-05,9999-01-01
8,10007,Senior Staff,1996-02-11,9999-01-01
9,10007,Staff,1989-02-10,1996-02-11


In [9]:
query = "select * from employee.departments limit 10;"
departments = pd.read_sql(query,connection)
departments

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing
5,d004,Production
6,d006,Quality Management
7,d008,Research
8,d007,Sales


In [10]:
query = "select * from employee.dept_emp limit 10;"
dept_emp = pd.read_sql(query,connection)
dept_emp

,emp_no,dept_no,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01
5,10006,d005,1990-08-05,9999-01-01
6,10007,d008,1989-02-10,9999-01-01
7,10008,d005,1998-03-11,2000-07-31
8,10009,d006,1985-02-18,9999-01-01
9,10010,d004,1996-11-24,2000-06-26


### 3. Introducción a Joins entre tablas:

#### Teoría de conjuntos: 
    Conjunto: 
        -Una colección de distintos elementos (Tsitsiklis).
        -Una agrupación, colección o reunión de objeto que cumplen una propiedad determinada.
    
    A los objetos del conjunto se denominan elementos. 
    
    
#### Ejemplos:
    1. El conjunto de estudiantes de data wrangling.
    2. El conjunto de letras de "data wrangling". 
    
#### Notación: 
    Los conjuntos se denotan con letras mayúsculas del alfabeto. Los elementos se escriben dentro de llaves.
    
    A = El conjunto de estudiantes de data wrangling. 
    B = El conjunto de letras de "data wrangling".
    
#### Determinación de conjuntos: 
    Extensión: se enumeran o se nombran los elementos del conjunto.
        B = {d,a,t,w,r,n,g,l} 
    Comprensión: Se enuncia la propiedad o la cualidad que distingue a los elementos.
        B = {x/ x es una letra de las palabras "data wrangling"}
    

#### Conjuntos especiales

        Conjunto vacío: aquel en el que ningún elemento cumple con la propiedad conocida como la regla de elegibilidad. 
        (Para cada familia de conjuntos, existe otro conjunto que contiene un elemento de cada uno de aquellos.)

    (Ver axioma de existencia) 

#### Propiedades de los conjuntos 

<img src="set_theory.png" />

- La intersección de conjuntos es conmutativa.
- La intersección de conjuntos es asociativa. 
- La intersección es distributiva respecto a la unión.
- La unión es distributiva respecto a la intersección.
- La unión de un conjunto con el universo es el universo.
- La intersección de un conjunto con su complemento es el conjunto vacío.



#### Joins entre tablas. 

Una de las aplicaciones más palpables de teoría de conjuntos en la ciencia de datos (aparte de teoría de probabilidad) es en manipulación y union entre sets de datos. De cierta manera, podemos pensar en nuestras colecciones de datos como conjuntos de elementos que comparten ciertas características en común. Sin embargo, estas características están dispersas en varias fuentes de información y parte previa al análisis es el proceso de consolidación, por lo cual se vuelve necesario realizar operaciones de conjuntos con ellos. 

Para esto, utilizaremos las keywords JOIN de SQL.


<img src="joins.jpg" />




### Ejemplo 1: A cada empleado, agregarle todos sus títulos y su vigencia.

In [12]:
%%time
### empleados y títulos 
query = """ SELECT
                A.emp_no,
                A.first_name,
                A.last_name,
                B.title,
                B.from_date,
                B.to_date
         FROM employee.employees A  
         LEFT JOIN employee.titles B on A.emp_no = B.emp_no
         LEFT JOIN 
         WHERE A.emp_no = 11080
"""
emp_titles = pd.read_sql(query,connection)
emp_titles

CPU times: user 7.24 ms, sys: 18.5 ms, total: 25.7 ms
Wall time: 581 ms


,emp_no,first_name,last_name,title,from_date,to_date
0,11080,Sanjai,Puppe,Engineer,1988-07-04,1993-07-04
1,11080,Sanjai,Puppe,Senior Engineer,1993-07-04,9999-01-01


### Ejercicio 1: 

Obtener el nombre, apellido,salario, titulo y vigencia del mismo para el empleado 12557. 
Su resultado se debe de ver de la siguiente manera:

<img src="ejercicio_1.png" />


In [23]:
query = """
SELECT
        
                A.first_name,
                A.last_name,
                A.emp_no,
                B.title,
                C.from_date,
                C.to_date,
                C.salary
         FROM employee.employees A  
          left join  employee.titles B on A.emp_no = B.emp_no
          left join employee.salaries C on A.emp_no = C.emp_no
         WHERE A.emp_no = 12557
"""
emp_12557 = pd.read_sql(query,connection)
emp_12557

,first_name,last_name,emp_no,title,from_date,to_date,salary
0,Stafford,Kopetz,12557,Senior Engineer,1988-03-20,1989-03-20,41840
1,Stafford,Kopetz,12557,Senior Engineer,1989-03-20,1990-03-20,46021
2,Stafford,Kopetz,12557,Senior Engineer,1990-03-20,1991-03-20,48699
3,Stafford,Kopetz,12557,Senior Engineer,1991-03-20,1992-03-19,51610
4,Stafford,Kopetz,12557,Senior Engineer,1992-03-19,1993-03-19,51351
5,Stafford,Kopetz,12557,Senior Engineer,1993-03-19,1994-03-19,51660
6,Stafford,Kopetz,12557,Senior Engineer,1994-03-19,1995-03-19,52878
7,Stafford,Kopetz,12557,Senior Engineer,1995-03-19,1996-03-18,54053
8,Stafford,Kopetz,12557,Senior Engineer,1996-03-18,1997-03-18,57950
9,Stafford,Kopetz,12557,Senior Engineer,1997-03-18,1998-03-18,58055


## Subqueries

### Ejemplo 2: Obtener el salario máximo por empleado.

### Paso 1: Armar un query con el salario máximo por empleado.

In [7]:
%%time
query = """
        SELECT
            emp_no
            ,MAX(salary) max_salary
        FROM employee.salaries
        GROUP BY 1
        LIMIT 100
        """
inner_table =pd.read_sql(query,connection)

CPU times: user 4.93 ms, sys: 4.05 ms, total: 8.98 ms
Wall time: 317 ms


### Paso 2: Estructura general de un subquery

In [10]:
####

query = """
        SELECT 
        B.first_name
        ,B.last_name, 
        A.* 
        FROM (        SELECT
            emp_no
            ,MAX(salary) max_salary
        FROM employee.salaries
        GROUP BY 1
        LIMIT 100) A
        LEFT JOIN employee.employees B on A.emp_no = B.emp_no 
        LIMIT 10
        """
subquery = pd.read_sql(query,connection)
subquery

,first_name,last_name,emp_no,max_salary
0,Georgi,Facello,10001,88958
1,Bezalel,Simmel,10002,72527
2,Parto,Bamford,10003,43699
3,Chirstian,Koblick,10004,74057
4,Kyoichi,Maliniak,10005,94692
5,Anneke,Preusig,10006,60098
6,Tzvetan,Zielinski,10007,88070
7,Saniya,Kalloufi,10008,52668
8,Sumant,Peac,10009,94443
9,Duangkaew,Piveteau,10010,80324


### Ejercicio 2: Encontrar el último departamento en el que trabajó el empleado con la información del mismo.

In [22]:
%%time
query = """
        SELECT 
        B.first_name
        ,B.last_name,
        B.birth_date,
        B.hire_date,
        C.dept_name,
        A.* 
        FROM (        SELECT
            emp_no,
            dept_no,
            MAX(from_date) fecha_max
        FROM employee.dept_emp
        GROUP BY 1,2
        LIMIT 100) A
        LEFT JOIN employee.employees B on A.emp_no = B.emp_no 
         LEFT JOIN employee.departments C on A.dept_no = C.dept_no 
        LIMIT 10
    

"""
pd.read_sql(query,connection)


CPU times: user 6.66 ms, sys: 4.29 ms, total: 11 ms
Wall time: 329 ms


,first_name,last_name,birth_date,hire_date,dept_name,emp_no,dept_no,fecha_max
0,Georgi,Facello,1953-09-02,1986-06-26,Development,10001,d005,1986-06-26
1,Bezalel,Simmel,1964-06-02,1985-11-21,Sales,10002,d007,1996-08-03
2,Parto,Bamford,1959-12-03,1986-08-28,Production,10003,d004,1995-12-03
3,Chirstian,Koblick,1954-05-01,1986-12-01,Production,10004,d004,1986-12-01
4,Kyoichi,Maliniak,1955-01-21,1989-09-12,Human Resources,10005,d003,1989-09-12
5,Anneke,Preusig,1953-04-20,1989-06-02,Development,10006,d005,1990-08-05
6,Tzvetan,Zielinski,1957-05-23,1989-02-10,Research,10007,d008,1989-02-10
7,Saniya,Kalloufi,1958-02-19,1994-09-15,Development,10008,d005,1998-03-11
8,Sumant,Peac,1952-04-19,1985-02-18,Quality Management,10009,d006,1985-02-18
9,Duangkaew,Piveteau,1963-06-01,1989-08-24,Production,10010,d004,1996-11-24


### Partition By 

In [5]:
query = """
    SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,C.dept_name
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
        LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
        GROUP BY 1,2,3,4,5,6
        LIMIT 100
"""
pd.read_sql(query,connection)

,emp_no,birth_date,first_name,last_name,gender,dept_name,seq
0,10001,1953-09-02,Georgi,Facello,M,Development,1
1,10002,1964-06-02,Bezalel,Simmel,F,Sales,1
2,10003,1959-12-03,Parto,Bamford,M,Production,1
3,10004,1954-05-01,Chirstian,Koblick,M,Production,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,Human Resources,1
...,...,...,...,...,...,...,...
95,10088,1954-02-25,Jungsoon,Syrzycki,F,Customer Service,1
96,10088,1954-02-25,Jungsoon,Syrzycki,F,Sales,2
97,10089,1963-03-21,Sudharsan,Flasterstein,F,Sales,1
98,10090,1961-05-30,Kendra,Hofting,M,Development,1


In [6]:
%%time
query = """
    SELECT *
    FROM (
        SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,C.dept_name
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
        LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
        GROUP BY 1,2,3,4,5,6
    ) A
    WHERE seq = 1
    LIMIT 100
"""
pd.read_sql(query,connection)

CPU times: user 6.37 ms, sys: 4.73 ms, total: 11.1 ms
Wall time: 2.38 s


,emp_no,birth_date,first_name,last_name,gender,dept_name,seq
0,10001,1953-09-02,Georgi,Facello,M,Development,1
1,10002,1964-06-02,Bezalel,Simmel,F,Sales,1
2,10003,1959-12-03,Parto,Bamford,M,Production,1
3,10004,1954-05-01,Chirstian,Koblick,M,Production,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,Human Resources,1
...,...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,M,Production,1
96,10097,1952-02-27,Remzi,Waschkowski,M,Research,1
97,10098,1961-09-23,Sreekrishna,Servieres,F,Customer Service,1
98,10099,1956-05-25,Valter,Sullins,F,Sales,1


### Ejercicio 3: Encontrar el salario máximo utilizando el Partition By

In [17]:
query = """

    SELECT *
    FROM ( 
    SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,B.salary
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.salary DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
        GROUP BY 1,2,3,4,5,6
    ) A
    WHERE seq = 1
    LIMIT 100
"""
pd.read_sql(query,connection)

,emp_no,birth_date,first_name,last_name,gender,salary,seq
0,10001,1953-09-02,Georgi,Facello,M,88958,1
1,10002,1964-06-02,Bezalel,Simmel,F,72527,1
2,10003,1959-12-03,Parto,Bamford,M,43699,1
3,10004,1954-05-01,Chirstian,Koblick,M,74057,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,94692,1
...,...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,M,68612,1
96,10097,1952-02-27,Remzi,Waschkowski,M,70161,1
97,10098,1961-09-23,Sreekrishna,Servieres,F,56202,1
98,10099,1956-05-25,Valter,Sullins,F,98538,1


### CASE WHEN 

In [ ]:
%%time
query = """
    SELECT
        YEAR(birth_date)
        ,SUM(CASE WHEN gender = 'M' THEN B.salary END) as Male_Salary
        ,SUM(CASE WHEN gender = 'F' THEN B.salary END) as Female_Salary
    FROM employee.employees A
    LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
    GROUP BY 1
"""
pd.read_sql(query,connection)


### Ejercicio 4: Usando la tabla del máximo salario, encontrar la suma de salarios por departamentos

In [ ]:
%%time
query = """

"""
pd.read_sql(query,connection)